In [ ]:
!wget --no-check-certificate \
  https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

In [ ]:
import zipfile
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
import os
base_dir = '/tmp/rockpaperscissors'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

rock_train_dir = os.path.join(train_dir, 'rock')
paper_train_dir = os.path.join(train_dir, 'paper')
scissors_train_dir = os.path.join(train_dir, 'scissors')

rock_validation_dir = os.path.join(validation_dir, 'rock')
paper_validation_dir = os.path.join(validation_dir, 'paper')
scissors_validation_dir = os.path.join(validation_dir, 'scissors')

In [ ]:
dirs = [train_dir, validation_dir, rock_train_dir, paper_train_dir, scissors_train_dir, rock_validation_dir, paper_validation_dir, scissors_validation_dir]
for dir in dirs:
    if not os.path.exists(dir):
        os.mkdir(dir)

In [ ]:
import shutil

rock_dir = f'{base_dir}/rock'
paper_dir = f'{base_dir}/paper'
scissors_dir = f'{base_dir}/scissors'

image_dirs = [rock_dir, paper_dir, scissors_dir]
for dir in image_dirs:
    file_list = os.listdir(dir)

    total_files = len(file_list)
    val_set = int(0.4 * total_files)

    for file_name in file_list[:val_set]:
        if dir.endswith("rock"):
            source_path = os.path.join(rock_dir, file_name)
            destination_path = os.path.join(rock_validation_dir, file_name)
            shutil.move(source_path, destination_path)
            print(f"Moved: {source_path} to {destination_path}")
        elif dir.endswith("paper"):
            source_path = os.path.join(paper_dir, file_name)
            destination_path = os.path.join(paper_validation_dir, file_name)
            shutil.move(source_path, destination_path)
            print(f"Moved: {source_path} to {destination_path}")
        elif dir.endswith("scissors"):
            source_path = os.path.join(scissors_dir, file_name)
            destination_path = os.path.join(scissors_validation_dir, file_name)
            shutil.move(source_path, destination_path)
            print(f"Moved: {source_path} to {destination_path}")

    for file_name in file_list[val_set:]:
        if dir.endswith("rock"):
            source_path = os.path.join(rock_dir, file_name)
            destination_path = os.path.join(rock_train_dir, file_name)
            shutil.move(source_path, destination_path)
            print(f"Moved: {source_path} to {destination_path}")
        elif dir.endswith("paper"):
            source_path = os.path.join(paper_dir, file_name)
            destination_path = os.path.join(paper_train_dir, file_name)
            shutil.move(source_path, destination_path)
            print(f"Moved: {source_path} to {destination_path}")
        elif dir.endswith("scissors"):
            source_path = os.path.join(scissors_dir, file_name)
            destination_path = os.path.join(scissors_train_dir, file_name)
            shutil.move(source_path, destination_path)
            print(f"Moved: {source_path} to {destination_path}")


In [ ]:
print(f"Total training rock images: {len(os.listdir(rock_train_dir))}")
print(f"Total training paper images: {len(os.listdir(paper_train_dir))}")
print(f"Total training scissors images: {len(os.listdir(scissors_train_dir))}")
print(f"Total training images: {len(os.listdir(rock_train_dir))+len(os.listdir(paper_train_dir))+len(os.listdir(scissors_train_dir))}")
print("===============================")
print(f"Total validation rock images: {len(os.listdir(rock_validation_dir))}")
print(f"Total validation paper images: {len(os.listdir(paper_validation_dir))}")
print(f"Total validation scissors images: {len(os.listdir(scissors_validation_dir))}")
print(f"Total validation images: {len(os.listdir(rock_validation_dir))+len(os.listdir(paper_validation_dir))+len(os.listdir(scissors_validation_dir))}")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
                    rescale=1.0/255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                #     zoom_range=0.2,
                #     width_shift_range=0.2,
                #     height_shift_range=0.2,
                    fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

In [ ]:
validation_datagen = ImageDataGenerator(
                    rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

In [ ]:
labels = {value: key for key, value in train_generator.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(15, 12))
idx = 0

for i in range(2):
    for j in range(5):
        label = labels[np.argmax(train_generator[0][1][idx])]
        ax[i, j].set_title(f"{label}")
        ax[i, j].imshow(train_generator[0][0][idx][:, :, :])
        ax[i, j].axis("off")
        idx += 1

plt.tight_layout()
plt.suptitle("Sample Training Images", fontsize=21)
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.regularizers import l2

def create_model():
    model = Sequential([
        Conv2D(filters=128, kernel_size=(5, 5), padding='valid', input_shape=(150, 150, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Conv2D(filters=64, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Conv2D(filters=32, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Flatten(),

        Dense(units=256, activation='relu'),
        Dropout(0.5),
        Dense(units=6, activation='softmax')
    ])

    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

model.compile(optimizer=Adam(learning_rate=0.001), loss=CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
history = model.fit(train_generator, epochs=50, validation_data=validation_generator,
                       verbose=2,
                       callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=5)])

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))

  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=10)
  print(fn)
  if classes==1:
   print('messy')
  else:
   print('clean')

In [ ]:
classes

In [ ]:
print(train_generator.class_indices)